In [1]:
import pandas as pd
import glob


# Brian Kitano Coding Test

Start: 9:23 AM

## 2. Data Cleaning

Start: 9:23 AM

The central task is to collapse the grid-level dataset into a district-level dataset.

The final product from this section is a district-level daily dataset from 2009-2013 with temperature, rainfall, and total rainfall variables.

Steps:
1. Calculate the set of points $P$ to include for each district.
2. Use the formulas to calculate the district statistic.

The actual EPIC project used 339 districts in India, 112 years of data and a 0.25°(latitude) x 0.25°(longitude) grid for rainfall. How would your code scale up with this larger dataset? Would you need any additional computing resources? Be as specific as possible.

### Response

Algorithm: weighted average of daily mean temp, mean rainfall, and total rainfall for all grid points w/in 100km of each district's geographic center. Weights are inverse of the squared distance from the district center.

$$
\bar t = \frac{1}{|P|} \sum_{p \in P} \frac{t_p}{(d - p)^2}
$$
$$
\bar r = \frac{1}{|P|} \sum_{p \in P} \frac{r_p}{(d - p)^2}
$$
$$
R = \sum_{p \in P} \frac{r_p}{(d - p)^2}
$$

where $P$ is the set of points within 100km of the district centroid, $d$ is the district centroid.


Assumptions:
- Are the `rainfall` csv's identical?

In [31]:
dd = pd.read_csv('./data/Rainfall/rainfall_2010.csv')
len(dd)

350765

In [4]:
# TODO: move to utils file
def mergeCsvsInDirectory(path):
    all_files = glob.glob(path + "/*.csv")

    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    mergedDf = pd.concat(li, axis=0, ignore_index=True)
    return mergedDf

In [5]:
raw_rain_df = mergeCsvsInDirectory('./data/Rainfall/')
raw_temp_df = mergeCsvsInDirectory('./data/Temperature/')


In order to do our analysis, we need to ensure that every entry has at least a day, month, year, latitude and longitude. Let's remove any that don't.

In [6]:

def getDefinedRows(df):
    rows_with_data = \
        pd.notna(df['latitude']) & \
        pd.notna(df['longitude']) & \
        pd.notna(df['day']) & \
        pd.notna(df['month']) & \
        pd.notna(df['year'])
    
    return rows_with_data

rain_df = raw_rain_df[getDefinedRows(raw_rain_df)]
temp_df = raw_temp_df[getDefinedRows(raw_temp_df)]

We'll want to create an `index` column in both dataframes so that we can join them. The index needs to be identical in both data frames. Note that the `date` column is formatted differently in each dataframe, but `day`, `month`, and `year` are the same. Our index will be `{lat}_{long}_{month}_{day}_{year}`.

In [7]:
def getIndex(df):
    indexColumn = \
    df["latitude"].astype(str) + '-' + \
    df["longitude"].astype(str) + '-' + \
    df["month"].astype(str) + '-' + \
    df["day"].astype(str) + '-' + \
    df["year"].astype(str)
    
    return indexColumn
    

The following approach won't scale well if there's a lot of years. 

In [40]:
len(temp_df) == len(temp_df)

True

In [18]:
on_columns = ['latitude', 'longitude', 'day', 'month', 'year']
total_df = rain_df.merge(temp_df, how='outer', left_on=on_columns, right_on=on_columns)

In [22]:
# sanity check
len(total_df) == len(rain_df) & len(total_df) == len(temp_df)

True

In [27]:
rainfall_values = sum(pd.notna(total_df['rainfall']))
print(f'rainfall values: {rainfall_values}')

temp_values = sum(pd.notna(total_df['temperature']))
print(f'temp values: {temp_values}')

rainfall values: 564234
temp values: 644062


In [28]:
total_df.to_csv('./total.csv')